In [1]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

def read_data():
    listings_data = pd.read_csv('listings.csv.gz')
    amenities_data = pd.read_json('amenities-vancouver.json.gz', lines=True)
    return listings_data, amenities_data


def clean_amenities_data(amenities_data, amenities_required):

    #find unique amenities and the number of them to choose which are important for a traveller
    # print(amenities_data['amenity'].value_counts())

    #adapted from : https://www.kite.com/python/answers/how-to-filter-a-pandas-dataframe-with-a-list-by-%60in%60-or-%60not-in%60-in-python
    bool_series = amenities_data.amenity.isin(amenities_required)
    filtered_amenities_df = amenities_data[bool_series]
    filtered_amenities_df = filtered_amenities_df.drop(['timestamp','tags'], axis=1).dropna() # dropping unnecessary columns, and filter out NA values
    filtered_amenities_df.reset_index(inplace=True, drop=True)

    return filtered_amenities_df


#reference: https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
def haversine_distance(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r * 1000

def haversine_distance2(df, lon2, lat2):
    # convert decimal degrees to radians 
    lon1=np.radians(df['lon'])
    lat1=np.radians(df['lat'])
    lon2=np.radians(lon2)
    lat2=np.radians(lat2)
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = (dlat/2).apply(sin)**2 + (lat1).apply(sin) * cos(lat2) * (dlon/2).apply(sin)**2
    c = 2 * ((a).apply(sqrt).apply(asin)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r * 1000

def clean_listings_data(listings_data):
    #keep only the columns we need
    columns_needed = ['id', 'listing_url', 'name', 'description', 'picture_url', 'latitude', 'longitude', 'property_type', 'accommodates', 'bedrooms', 'beds', 'amenities', 'price']
    listings_data = listings_data[columns_needed]
    return listings_data


# #TODO: return a dictionary with number of amenities in a 1km radius of this lat and lon
def num_amenities(lat, lon, amenities_data_clean):
    distance = haversine_distance2(amenities_data_clean, lon, lat)
    amenities_data_clean['distance'] = distance
    data_withinR = amenities_data_clean.loc[amenities_data_clean['distance'] < 1000].reset_index(drop=True)
    amenities_series = data_withinR.pivot_table(columns = ['amenity'], aggfunc='size')  # Counts # of amenities, type=pd.series
    amenities_dict = amenities_series.to_dict()# converts series to dict
    return amenities_dict

In [2]:
#Read Data
listings_data, amenities_data = read_data()

# Change amenities here (updated the "restaurant" typo)
amenities_required = ['restaurant', 'fast_food', 'cafe','bank','atm','pharmacy','bicycle_rental','fuel','pub','bar','car_sharing','car_rental','clinic','doctors','hospital','ice_cream','fountain','theatre','police','bus_station']

#Data Cleaning
amenities_data_clean = clean_amenities_data(amenities_data, amenities_required)
listings_data_clean = clean_listings_data(listings_data)


#Return a dict of amenities:
lat_input = 49.225164  # sample lat input
lon_input = -123.003742  # sample lon input
amen_dict = num_amenities(lat_input, lon_input, amenities_data_clean)

In [3]:
amen_dict

{'bank': 16,
 'bar': 16,
 'bus_station': 1,
 'cafe': 27,
 'car_rental': 1,
 'clinic': 8,
 'fast_food': 61,
 'fountain': 1,
 'fuel': 1,
 'ice_cream': 1,
 'pharmacy': 9,
 'police': 1,
 'pub': 1,
 'restaurant': 56}

In [5]:
listings_data_clean = listings_data_clean.head(5)
listings_data_clean

,id,listing_url,name,description,picture_url,latitude,longitude,property_type,accommodates,bedrooms,beds,amenities,price
0,10080,https://www.airbnb.com/rooms/10080,D1 - Million Dollar View 2 BR,"Stunning two bedroom, two bathroom apartment. ...",https://a0.muscache.com/pictures/55778229/c2f7...,49.28872,-123.12046,Entire condominium,5,2.0,2.0,"[""Iron"", ""Dryer"", ""Bed linens"", ""Cooking basic...",$150.00
1,13358,https://www.airbnb.com/rooms/13358,Monthly (or Longer ) Designer One Bedroom Down...,<b>The space</b><br />This suites central loca...,https://a0.muscache.com/pictures/c23bb7ef-e300...,49.28201,-123.12669,Entire condominium,2,1.0,1.0,"[""Iron"", ""Dryer"", ""Bed linens"", ""Cooking basic...",$85.00
2,13490,https://www.airbnb.com/rooms/13490,Vancouver's best kept secret,This apartment rents for one month blocks of t...,https://a0.muscache.com/pictures/73394727/79d5...,49.25622,-123.06607,Entire apartment,2,1.0,1.0,"[""Iron"", ""Outdoor dining area"", ""Ethernet conn...",$145.00
3,14267,https://www.airbnb.com/rooms/14267,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",https://a0.muscache.com/pictures/3646de9b-934e...,49.24922,-123.08139,Entire house,4,1.0,2.0,"[""Iron"", ""Dryer"", ""Cooking basics"", ""Hot water...",$140.00
4,16254,https://www.airbnb.com/rooms/16254,Close to PNE/Hastings Park and East Village,2 Bedroom garden level guest suite.(lower leve...,https://a0.muscache.com/pictures/90623667/583c...,49.27721,-123.04086,Entire guest suite,4,2.0,3.0,"[""Iron"", ""Dryer"", ""Bed linens"", ""Cooking basic...",$195.00


In [6]:
listings_data_clean['num_amenities_nearby'] = listings_data_clean.apply(lambda x: num_amenities(x['latitude'], x['longitude'], amenities_data_clean), axis = 1)

In [8]:
listings_data_clean['num_amenities_nearby']

0    {'atm': 1, 'bank': 21, 'bar': 9, 'bicycle_rent...
1    {'atm': 1, 'bank': 30, 'bar': 27, 'bicycle_ren...
2    {'bank': 4, 'bicycle_rental': 7, 'cafe': 7, 'c...
3    {'bank': 3, 'bicycle_rental': 2, 'cafe': 7, 'c...
4    {'bank': 4, 'cafe': 6, 'doctors': 4, 'fast_foo...
Name: num_amenities_nearby, dtype: object